In [22]:
import numpy as np
import mnist_loader

In [28]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        # bias的数量和该层的神经元的数量一致，第一层是输入层，无bias
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        # 每层weight的shape是当前层和上一层神经元数量的结合
        # wji表示当前层的第j和神经元和上一层的第i个神经元之间的权重
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
    
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w,a)+b)
        return a
    
    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The "mini_batch" is a list of tuples "(x, y)", and "eta"
        is the learning rate."""
        # 设置b和w的gradient base,每一轮会往这个base上面加上一个gradient，然后我们会用初始的w,b不断减去这个base来进行优化
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            # backprop是进行求偏导的关键算法
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        # 因为loss function为 1/n*(||y - y^||2)所以n要算在里面，而mini-batch不为n,数量为m，所以要添加进去。暂时解释不清楚
        self.weights = [w-(eta/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]

    def SGD(self, training_data, epochs, mini_batch_size, eta,test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The "training_data" is a list of tuples
        "(x, y)" representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If "test_data" is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            np.random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)
                
                
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)


def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [30]:
training_data, validation_data, test_data =  mnist_loader.load_data_wrapper()

In [38]:
net = Network([784, 100, 10])

In [41]:
net.SGD(training_data, 40, 10, 3.0, test_data=test_data)

Epoch 0: 9487 / 10000
Epoch 1: 9515 / 10000
Epoch 2: 9509 / 10000
Epoch 3: 9523 / 10000
Epoch 4: 9524 / 10000
Epoch 5: 9507 / 10000
Epoch 6: 9504 / 10000
Epoch 7: 9533 / 10000
Epoch 8: 9534 / 10000
Epoch 9: 9530 / 10000
Epoch 10: 9551 / 10000
Epoch 11: 9550 / 10000
Epoch 12: 9531 / 10000
Epoch 13: 9556 / 10000
Epoch 14: 9553 / 10000
Epoch 15: 9564 / 10000
Epoch 16: 9561 / 10000
Epoch 17: 9602 / 10000
Epoch 18: 9657 / 10000
Epoch 19: 9647 / 10000
Epoch 20: 9649 / 10000
Epoch 21: 9650 / 10000
Epoch 22: 9638 / 10000
Epoch 23: 9645 / 10000
Epoch 24: 9663 / 10000
Epoch 25: 9664 / 10000
Epoch 26: 9666 / 10000
Epoch 27: 9657 / 10000
Epoch 28: 9657 / 10000
Epoch 29: 9654 / 10000
Epoch 30: 9668 / 10000
Epoch 31: 9669 / 10000
Epoch 32: 9662 / 10000
Epoch 33: 9662 / 10000
Epoch 34: 9664 / 10000
Epoch 35: 9662 / 10000
Epoch 36: 9666 / 10000
Epoch 37: 9665 / 10000
Epoch 38: 9662 / 10000
Epoch 39: 9669 / 10000


# Using cross-entropy cost function and other techs

In [42]:
import network2

In [46]:
net2 = network2.Network([784, 30, 10], cost=network2.CrossEntropyCost)
net2.large_weight_initializer()
net2.SGD(training_data, 30, 10, 0.5, evaluation_data=test_data, monitor_evaluation_accuracy=True)

Epoch 0 training complete
Accuracy on evaluation data: 9119 / 10000

Epoch 1 training complete
Accuracy on evaluation data: 9310 / 10000

Epoch 2 training complete
Accuracy on evaluation data: 9315 / 10000

Epoch 3 training complete
Accuracy on evaluation data: 9387 / 10000

Epoch 4 training complete
Accuracy on evaluation data: 9396 / 10000

Epoch 5 training complete
Accuracy on evaluation data: 9427 / 10000

Epoch 6 training complete
Accuracy on evaluation data: 9477 / 10000

Epoch 7 training complete
Accuracy on evaluation data: 9468 / 10000

Epoch 8 training complete
Accuracy on evaluation data: 9484 / 10000

Epoch 9 training complete
Accuracy on evaluation data: 9465 / 10000

Epoch 10 training complete
Accuracy on evaluation data: 9465 / 10000

Epoch 11 training complete
Accuracy on evaluation data: 9510 / 10000

Epoch 12 training complete
Accuracy on evaluation data: 9456 / 10000

Epoch 13 training complete
Accuracy on evaluation data: 9514 / 10000

Epoch 14 training complete
Acc

([],
 [9119,
  9310,
  9315,
  9387,
  9396,
  9427,
  9477,
  9468,
  9484,
  9465,
  9465,
  9510,
  9456,
  9514,
  9482,
  9485,
  9506,
  9476,
  9497,
  9492,
  9501,
  9497,
  9512,
  9510,
  9489,
  9502,
  9488,
  9515,
  9502,
  9501],
 [],
 [])